# PFHub Benchmark 6 v2

This repository implements the [Cahn-Hilliard-Poisson Benchmark v2][chpb]
from [PFHub][pfhb] using [MMSP][mmsp] with a [convex splitting][cnvx] formulation.

While legible on GitHub, this notebook is best viewed on [nbviewer.ipython.org][ipnb].

<!--References-->
[chpb]: https://pages.nist.gov/pfhub/benchmarks/benchmark6-hackathon.ipynb/
[cnvx]: https://doi.org/10.1557/PROC-529-39
[ipnb]: http://nbviewer.jupyter.org/github/tkphd/pfhub-bm6-cahn-hilliard-poisson/blob/master/README.ipynb
[mmsp]: https://github.com/mesoscale/mmsp
[pfhb]: https://pages.nist.gov/pfhub

## Equations of Motion

The EOMs are provided as follows:

In [1]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

In [2]:
%%latex
\begin{align}
\frac{\partial c}{\partial t} &= M'(c)\nabla c\nabla\mu + M(c)\nabla^2\mu\\
\mu &= 4wc^3 - 6w(c_{\alpha} + c_{\beta})c^2 + 2w(c_{\alpha}^2 + 4c_{\alpha}c_{\beta} + c_{\beta}^2)c - 2wc_{\alpha}c_{\beta}(c_{\alpha} + c_{\beta}) - \kappa\nabla^2c + k\Phi_{\mathrm{tot}}\\
\nabla^2\Phi_{\mathrm{tot}} &= \frac{-k}{\epsilon}(c - c_0)
\end{align}
where $M(c) = \frac{M_0}{1+c^2}$ and $M'(c) = \frac{-2M_0c}{(1+c^2)^2}$.

<IPython.core.display.Latex object>

## Discretizations

Equations (1)--(3) are discretized as follows, using the subscript $n$ to indicate the current timestep and $n+1$ indicating the next timestep.

### Composition

In [3]:
%%latex

Linearizing Equation (1):

\begin{equation}
c_{n+1} = c_n + \Delta t\left[\frac{-2M_0\nabla c_n\cdot\nabla\mu_n}{(1+c_n^2)^2}\right]c_{n+1}
               + \Delta t M(c_n)\nabla^2\mu_{n+1}\\
\end{equation}

Grouping update terms ($n+1$) on the left,

\begin{equation}
\left[1 + \frac{2M_0\Delta t\nabla c_n\cdot\nabla\mu_n}{(1+c_n^2)^2}\right]c_{n+1} - \Delta t M(c_n)\nabla^2\mu_{n+1} = c_n
\end{equation}

The 2-D discrete Laplacian operator ($\nabla^2$) can be separated into on-diagonal and off-diagonal terms, assuming $\Delta x = \Delta y = h$:
    
\begin{equation}
\nabla^2\eta = \frac{\eta_{i+1,j} + \eta_{i-1,j}}{h^2} + \frac{\eta_{i,j+1} + \eta_{i,j-1}}{h^2} - \frac{4\eta_{i,j}}{h^2} = \nabla^2_{\LARGE\circ}\eta_n - \frac{4}{h^2}\eta_{n+1}
\end{equation}

In the implementation, $\nabla^2_{\circ}$ is referred to as the "fringe Laplacian,"
as it takes values from the fringe of the stencil around the central value (but not the central value itself).

<IPython.core.display.Latex object>

In [4]:
%%latex

Substituting Eqn. (6) into Eqn. (5), and letting $\xi = \frac{4}{h^2}$, we arrive at the convex splitting discretization of Eqn. (1):

\begin{equation}
\left[1 + \frac{2M_0\Delta t\nabla c_n\cdot\nabla\mu_n}{(1+c_n^2)^2}\right]c_{n+1} 
        + \Delta t \xi M(c_n)\mu_{n+1} = c_n + \Delta t M(c_n)\nabla^2_{\LARGE\circ}\mu_n.
\end{equation}

<IPython.core.display.Latex object>

### Chemical Potential

In [5]:
%%latex

Eqn. (2) is derived from a free energy expression with both convex and non-convex terms.
To first order, let us assume that terms in Eqn. (2) with odd powers derive from convex
(contractive) terms, and those with even powers derive from non-convex (expansive) terms.
Then the convex terms are evaluated at $n+1$, and the rest at $n$.

\begin{equation}
\mu = \underbrace{2w\left(2c^2 + (c_{\alpha}^2 + 4c_{\alpha}c_{\beta} + c_{\beta}^2)\right)c}_{\mathrm{contractive}} 
    + \underbrace{2w\left(-3(c_{\alpha} + c_{\beta})c^2 - c_{\alpha}c_{\beta}(c_{\alpha} + c_{\beta})\right)}_{\mathrm{expansive}}
    - \kappa\nabla^2c + k\Phi + k\Phi_{\mathrm{ext}}
\end{equation}

<IPython.core.display.Latex object>

In [6]:
%%latex

Eqn. (8) can then be linearized and regrouped:

\begin{align}
f_{\mathrm{con}}(c) &= 2w\left(2c^2 + (c_{\alpha}^2 + 4c_{\alpha}c_{\beta} + c_{\beta}^2)\right)\\
f_{\mathrm{exp}}(c) &= 2w\left(-3(c_{\alpha} + c_{\beta})c^2 - c_{\alpha}c_{\beta}(c_{\alpha} + c_{\beta})\right)
\end{align}

\begin{equation}
-\left[f_{\mathrm{con}}(c_n) + \kappa\xi\right]c_{n+1} + \mu_{n+1} - k\Phi_{n+1} = f_{\mathrm{exp}}(c_n) -\kappa\nabla^2_{\LARGE\circ}c_n + k\Phi_{\mathrm{ext}}
\end{equation}


<IPython.core.display.Latex object>

### Electrostatic Potential

In [7]:
%%latex

Discretizing Eqn. (3) is relatively straightforward, since $\nabla^2\Phi_{\mathrm{ext}}\equiv 0$:

\begin{align}
\nabla^2\Phi_{\mathrm{tot}} &= \frac{-k}{\epsilon}(c - c_0)\\
\nabla^2\Phi_{n+1} &= -\frac{kc}{\epsilon} + \frac{kc_0}{\epsilon}\\
-\xi\Phi_{n+1} + \frac{k}{\epsilon}c_{n+1} &= \frac{kc_0}{\epsilon} - \nabla^2_{\LARGE\circ}\Phi_{n}
\end{align}

<IPython.core.display.Latex object>

## Matrix Form

In [8]:
%%latex

Equations (7), (11), and (14) form the system of equations to solve. In matrix form, this becomes

\begin{equation}
\left[\begin{array}\\
1 + M'(c_n)\Delta t \nabla c_n\cdot\nabla\mu_n  & M(c_n)\xi\Delta t & 0\\
-\left[f_{\mathrm{con}}(c_n) + \kappa\xi\right] & 1                 & -k\\
\frac{k}{\epsilon}                              & 0                 & -\xi\\
      \end{array}\right] \left[\begin{array}\\
c_{n+1}\\ \mu_{n+1}\\ \Phi_{n+1}\end{array}\right] = \left[\begin{array}\\
c_n + M(c_n)\Delta t \nabla^2_{\LARGE\circ}\mu_n\\
f_{\mathrm{exp}}(c_n) - \kappa\nabla^2_{\LARGE\circ}c_n + k\Phi_{\mathrm{ext}}\\
\frac{kc_0}{\epsilon} - \nabla^2_{\LARGE\circ}\Phi_n\\
\end{array}\right]
      
\end{equation}

<IPython.core.display.Latex object>

Eqn. (15) is vulnerable to [Cramer's rule][crmr].

<!--References-->
[crmr]: https://en.wikipedia.org/wiki/Cramer%27s_rule#Explicit_formulas_for_small_systems

In [9]:
%%latex

\begin{align}
\mathrm{det}(A)  &= (a_{11}a_{22}a_{33} - 0) + (a_{12}a_{23}a_{31} - a_{12}a_{21}a_{33}) + (0 - 0)\\
                 &= a_{33}(a_{11}a_{22} - a_{12}a_{21}) + a_{12}a_{23}a_{31}.\\
\mathrm{det}(A_1) &= (b_{1}a_{22}a_{33} - 0) + (a_{12}a_{23}b_{3} - a_{12}b_{2}a_{33}) + (0 - 0)\\
                 &= a_{33}(b_{1}a_{22} - a_{12}b_{2}) + a_{12}a_{23}b_{3}.\\
\mathrm{det}(A_2) &= (a_{11}b_{2}a_{33} - a_{11}a_{23}b_{3}) + (b_{1}a_{23}a_{31} - b_{1}a_{21}a_{33}) + (0 - 0)\\
                 &= a_{33}(a_{11}b_{2} - b_{1}a_{21}) + a_{23}(b_{1}a_{31} - a_{11}b_{3}).\\
\mathrm{det}(A_3) &= (a_{11}a_{22}b_{3} - 0) + (a_{12}b_{2}a_{31} - a_{12}a_{21}b_{3}) + (0 - b_{1}a_{22}a_{31})\\
                 &= a_{22}(a_{11}b_{3} - b_{1}a_{31}) + a_{21}(b_{2}a_{31} - a_{21}b_{3}).\\
c_{n+1}    &= \frac{\mathrm{det}(A_1)}{\mathrm{det}(A)}\\
\mu_{n+1}  &= \frac{\mathrm{det}(A_2)}{\mathrm{det}(A)}\\
\Phi_{n+1} &= \frac{\mathrm{det}(A_3)}{\mathrm{det}(A)}\\
\end{align}

<IPython.core.display.Latex object>